使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第七節 温水暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import numpy as np
from s04_07_common import get_Q_out_H_hs_d_t
from s04_08_a_heatpump_cycle import calc_e_ref_H_th
import s04_07_h_max_capacity_of_hot_water_floor_heating as appendix_H

## 付録N 地中熱ヒートポンプ温水暖房機 / Appendix N Geothermal heat pump hot water heater

### N.1 適用範囲 / Scope of application

省略

ommision

### N.2 記号及び単位 / Symbol and unit

省略

ommision

### N.3 暖房エネルギー消費量 / Heating energy consumption

#### N.3.1 消費電力量 / Power consumption

$$\large
E_{E,hs,d,t} = \left\{ \begin{array}{ll}
    E_{comp,hs,d,t}+E_{pump,hs,d,t}+E_{aux,hs,d,t} &(q_{out,H,hs,d,t}\neq0)\\ 
    0 &(q_{out,H,hs,d,t}=0)\\ 
    \end{array}
\right.
$$

$E_{aux,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの補機の消費電力量(kWh/h) / Electric power consumption of auxiliary equipment per hour at time $t$ on date $d$  
$E_{comp,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの圧縮機の消費電力量(kWh/h) / Compressor power consumption per hour at time $t$ on date $d$  
$E_{E,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの温水暖房用熱源機の消費電力量(kWh/h) / Electricity consumption of the heat source machine for hot water heating per hour at time $t$ on date $d$  
$E_{pump,hs,d,t}$：日付$d$の時刻$t$における1 時間当たりのポンプの消費電力量(kWh/h) / Hourly pump power consumption at time $t$ on date $d$  

In [ ]:
def calc_E_E_hs_d_t(Q_dmd_H_hs_d_t, Theta_ex_a_Ave, Theta_ex_d_Ave_d, Theta_ex_H_Ave, Theta_SW_d_t, q_max_hs, L_H_x_t_i, L_CS_x_t_i, L_CL_x_t_i, HeatExchangerType):
    """ 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の消費電力量 (1)

    :param Q_dmd_H_hs_d_t: 1時間当たりの熱源機の熱需要 (MJ/h)
    :type Q_dmd_H_hs_d_t: ndarray
    :param Theta_ex_a_Ave: 年平均外気温度 (℃)
    :type Theta_ex_a_Ave: float
    :param Theta_ex_d_Ave_d: 日付dにおける日平均外気温度 (℃)
    :type Theta_ex_d_Ave_d: ndarray
    :param Theta_ex_H_Ave: 暖房期における期間平均外気温度（℃）
    :type Theta_ex_H_Ave: float
    :param Theta_SW_d_t: 往き温水温度 (℃)
    :type Theta_SW_d_t: ndarray
    :param q_max_hs: 熱源機の最大暖房能力 ⒲
    :type q_max_hs: float
    :param L_H_x_t_i: 暖冷房区画iの1時間当たりの暖房負荷
    :type L_H_x_t_i: ndarray
    :param L_CS_x_t_i: 日付dの時刻tにおける暖冷房区画iの冷房顕熱負荷 (MJ/h)
    :type L_CS_x_t_i: ndarray
    :param L_CL_x_t_i: 日付dの時刻tにおける暖冷房区画iの冷房潜熱負荷 (MJ/h)
    :type L_CL_x_t_i: ndarray
    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 1時間当たりの熱源機の消費電力量 (kWh/h)
    :rtype: ndarray
    """
    # ---------- 地中熱交換器からの戻り熱源水の日平均温度 ----------

    # 1日当たりの冷房全熱負荷の年間最大値（MJ/d）(20c)
    L_max_C = get_L_max_C(L_CS_x_t_i, L_CL_x_t_i)

    # 1日当たりの暖房負荷の年間最大値（MJ/d）(20b)
    L_max_H = get_L_max_H(L_H_x_t_i)

    # 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）(20a)
    R_L_max = get_R_L_max(L_max_H, L_max_C)

    # 地中熱交換器からの戻り熱源水温度を求める式の係数（-）(19)
    K_gsRW_H = calc_K_gsRW_H(R_L_max, HeatExchangerType)

    # 暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差（℃）(18)
    Delta_Theta_gsRW_H = calc_Delta_Theta_gsRW_H(R_L_max, HeatExchangerType)

    # 日付dにおける地中熱交換器からの戻り熱源水の日平均温度（℃）(17)
    Theta_gsRW_d_ave_d = get_Theta_gsRW_d_ave_d(K_gsRW_H, Theta_ex_d_Ave_d, Theta_ex_H_Ave, Theta_ex_a_Ave, Delta_Theta_gsRW_H)

    # ---------- 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力 ----------

    # 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の最大暖房出力（MJ/h）(16)
    q_max_H_hs_JRA = calc_q_max_H_hs_JRA(q_max_hs)

    # ---------- 温水暖房用熱源機内の平均放熱損失 ----------

    # 日付dの時刻tにおける温水暖房用の熱源機内部の平均放熱損失 (kw) (N.9)
    q_loss_H_hs_d_t = get_q_loss_H_hs_d_t()

    # ---------- 温水暖房用熱源機の平均暖房出力 ----------

    # 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の最大暖房出力（MJ/h）(15)
    Q_max_H_hs_d_t = calc_Q_max_H_hs_d_t(Theta_SW_d_t, Theta_ex_d_Ave_d, Theta_ex_H_Ave, Theta_ex_a_Ave, q_max_hs, L_H_x_t_i, L_CS_x_t_i, L_CL_x_t_i, HeatExchangerType)

    # 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の暖房出力（MJ/h）(14)
    Q_out_H_hs_d_t = get_Q_out_H_hs_d_t(Q_dmd_H_hs_d_t, Q_max_H_hs_d_t)

    # 日付dの時刻tにおける温水暖房用熱源機の平均暖房出力(kW) (13)
    q_out_H_hs_d_t = get_q_out_H_hs_d_t(Q_out_H_hs_d_t)

    # ---------- 温水暖房用熱源機の最大暖房能力に対する平均負荷率 ----------

    # 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-) (12)
    qr_out_H_hs_d_t = get_qr_out_H_hs_d_t(q_out_H_hs_d_t, q_max_H_hs_JRA)

    # ---------- 補機の消費電力量 ----------

    # 日付dの時刻tにおける1時間当たりの補機の消費電力量(kWh/h) (11a)
    E_aux_hs_d_t = calc_E_aux_hs_d_t(qr_out_H_hs_d_t)

    # ---------- ポンプの消費電力量 ----------

    # 日付dの時刻tにおける1時間当たりの熱源水ポンプの消費電力量 (kWh/h) (10a)
    E_pump_gsRW_d_t = calc_E_pump_gsRW_d_t(qr_out_H_hs_d_t)

    # 日付dの時刻tにおける1時間当たりの送水ポンプの消費電力量 (kWh/h) (9a)
    E_pump_SW_d_t = calc_E_pump_SW_d_t(qr_out_H_hs_d_t)

    # 日付dの時刻tにおける1時間当たりのポンプの消費電力量 (kWh/h) (8)
    E_pump_hs_d_t = get_E_pump_hs_d_t(E_pump_SW_d_t, E_pump_gsRW_d_t)

    # ---------- 圧縮機の消費電力量 ----------

    # 日付dの時刻tにおけるヒートポンプサイクルの凝縮温度（℃）(7a)
    Theta_ref_SH_d_t = calc_Theta_ref_SH_d_t(qr_out_H_hs_d_t, Theta_gsRW_d_ave_d)

    # 日付dの時刻tにおけるヒートポンプサイクルの蒸発温度（℃）(6a)
    Theta_ref_SC_d_t = calc_Theta_ref_SC_d_t(qr_out_H_hs_d_t, Theta_SW_d_t)

    # 日付dの時刻tにおけるヒートポンプサイクルの凝縮温度（℃）(5a)
    Theta_ref_cnd_d_t = calc_Theta_ref_cnd_d_t(qr_out_H_hs_d_t, Theta_SW_d_t)

    # 日付dの時刻tにおけるヒートポンプサイクルの蒸発温度（℃）(4a)
    Theta_ref_evp_d_t = calc_Theta_ref_evp_d_t(qr_out_H_hs_d_t, Theta_gsRW_d_ave_d)

    # 日付dの時刻tにおける1時間当たりの圧縮機の圧縮効率 (-) (3a)
    Mu_d_t = calc_Mu_d_t(Theta_ref_evp_d_t, Theta_ref_cnd_d_t, Theta_ref_SC_d_t, Theta_ref_SH_d_t)

    # 日付dの時刻tにおける1時間当たりの圧縮機の消費電力量 (kWh/h) (2)
    E_comp_hs_d_t = get_E_comp_hs_d_t(qr_out_H_hs_d_t, q_out_H_hs_d_t, q_loss_H_hs_d_t, Mu_d_t, Theta_ref_evp_d_t, Theta_ref_cnd_d_t)

    # ---------- 熱源機の消費電力量 ----------

    # 1時間当たりの熱源機の消費電力量 (kWh/h) (1)
    E_E_hs_d_t = E_comp_hs_d_t + E_pump_hs_d_t + E_aux_hs_d_t
    E_E_hs_d_t[q_out_H_hs_d_t == 0] = 0

    return E_E_hs_d_t

#### N.3.2 ガス消費量 / Gas consumption

1 時間当たりの温水暖房用熱源機のガス消費量$E_{G,hs,d,t}$は、0とする。

The gas consumption $E_{G,hs,d,t}$ of the heat source unit for hot water heating per hour is set to 0.

In [ ]:
def get_E_G_hs_d_t():
    """ 熱源機のガス消費量

    :return: 熱源機のガス消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)

#### N.3.3 灯油消費量 / Kerosene consumption

1 時間当たりの温水暖房用熱源機の灯油消費量$E_{K,hs,d,t}$は、0とする。

The kerosene consumption $E_{K,hs,d,t}$ of the heat source unit for hot water heating per hour is 0.

In [ ]:
def get_E_K_hs_d_t():
    """ 熱源機の灯油消費量

    :return: 熱源機の灯油消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)

#### N.3.4 その他の一次エネルギー消費量 / Other primary energy consumption

1 時間当たりの温水暖房用熱源機のその他の一次エネルギー消費量$E_{M,hs,d,t}$は、0とする。

Other primary energy consumption $E_{M,hs,d,t}$ of the heat source unit for hot water heating per hour shall be 0.

In [ ]:
def get_E_M_hs_d_t():
    """ 熱源機のその他の燃料の一次エネルギー消費量

    :return: 熱源機のその他の燃料の一次エネルギー消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)

### N.4 圧縮機の消費電力量 / Compressor power consumption

$$\large
E_{comp,hs,d,t} = \left\{ \begin{array}{ll}
    f_{comp,act,d,t} \times \frac{q_{out,H,hs,d,t} + q_{loss,H,hs,d,t}}{\eta_{d,t}} &(\theta_{ref,evp,d,t}< \theta_{ref,cnd,d,t})\\  
    0 &(\theta_{ref,evp,d,t} \geq \theta_{ref,cnd,d,t})\\ 
    \end{array}
\right.
$$

$$\large
f_{comp,act,d,t} = max(k_{comp,a}\times qr_{out,H,hs,d,t})+(1-k_{comp,a}\times k_{comp,b}),1)\times k_{comp,c}
$$

$$\large
\left\{ \begin{array}{ll}
    k_{comp,a}= −0.7309\\  
    k_{comp,b}= 0.67\\  
    k_{comp,c}= 1.0319\\  
    \end{array}
\right.
$$

$E_{comp,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの圧縮機の消費電力量(kWh/h) / Compressor power consumption per hour at time $t$ on date $d$  
$f_{comp,act,d,t}$：日付$d$の時刻$t$における圧縮機の消費電力に対する補正係数(-) / Correction factor for compressor power consumption at time $t$ on date $d$  
$q_{loss,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機内部の平均放熱損失(kW) / Average heat radiation loss inside the heat source for hot water heating at time $t$ on date $d$  
$q_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の平均暖房出力(kW) / Average heating output of heat source machine for hot water heating at time $t$ on date $d$  
$qr_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の最大暖房能力に対する平均負荷率(-) / Average load factor for maximum heating capacity of the heat source unit for hot water heating at time $t$ on date $d$  
$\eta_{d,t}$：日付$d$の時刻$t$における圧縮機の圧縮効率(-) / Compression efficiency of compressor at time $t$ on date $d$  
$\theta_{ref,cnd,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの凝縮温度(℃) / Condensation temperature of heat pump cycle at time $t$ on date $d$  
$\theta_{ref,evp,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの蒸発温度(℃) / Evaporation temperature of heat pump cycle at time $t$ on date $d$  
$k_{comp,a},k_{comp,b},k_{comp,c}$：圧縮機の消費電力に対する補正係数を計算する式の係数(-) / Coefficient of the formula for calculating the correction coefficient for the power consumption of the compressor  

In [ ]:
def get_E_comp_hs_d_t(qr_out_H_hs_d_t, q_out_H_hs_d_t, q_loss_H_hs_d_t, Mu_d_t, Theta_ref_evp_d_t, Theta_ref_cnd_d_t):
    """ 日付dの時刻tにおける1時間当たりの圧縮機の消費電力量 (2)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :param q_out_H_hs_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type q_out_H_hs_d_t: ndarray
    :param q_loss_H_hs_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type q_loss_H_hs_d_t: ndarray
    :param Mu_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type Mu_d_t: ndarray
    :param Theta_ref_evp_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type Theta_ref_evp_d_t: ndarray
    :param Theta_ref_cnd_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type Theta_ref_cnd_d_t: ndarray
    :return: 日付dの時刻tにおける1時間当たりの圧縮機の消費電力量
    :rtype: ndarray
    """
    # 圧縮機の消費電力に対する補正係数を計算する式の係数(-) (2c)
    k_comp_a = -0.7309
    k_comp_b = 0.67
    k_comp_c = 1.0319

    f_comp_act_d_t = np.clip(k_comp_a * qr_out_H_hs_d_t + (1 - k_comp_a * k_comp_b), 1, None) * k_comp_c

    # 日付dの時刻tにおける1時間当たりの圧縮機の消費電力量 (2a)
    E_comp_hs_d_t = f_comp_act_d_t * ((q_out_H_hs_d_t + q_loss_H_hs_d_t) / Mu_d_t)
    E_comp_hs_d_t[Theta_ref_evp_d_t >= Theta_ref_cnd_d_t] = 0

    return E_comp_hs_d_t

$$\large
\eta_{d,t} = k_{\eta h 2} \times e^2_{ref,H,th,d,t} + k_{\eta h 1} \times e_{ref,H,th,d,t} + k_{\eta h 0}
$$

$$\large
\left\{ \begin{array}{ll}
    k_{\eta h 0}= −0.430363368361459\\  
    k_{\eta h 1}= 0.698531770387591\\  
    k_{\eta h 2}= 0.0100164335768507\\  
    \end{array}
\right.
$$

$\eta_{d,t}$：日付$d$の時刻$t$における圧縮機の圧縮効率(-) / Compression efficiency of compressor at time $t$ on date $d$  
$e_{ref,H,th,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの理論暖房効率(-) / Theoretical heating efficiency of heat pump cycle at time $t$ on date $d$  
$k_{\eta h 2},k_{\eta h 1},k_{\eta h 0}$：圧縮機の圧縮効率を求める式の係数(-) / Coefficient of the formula for calculating the compression efficiency of the compressor  

In [ ]:
def calc_Mu_d_t(Theta_ref_evp_d_t, Theta_ref_cnd_d_t, Theta_ref_SC_d_t, Theta_ref_SH_d_t):
    """ 日付dの時刻tにおける圧縮機の圧縮効率 (3a)

    :param Theta_ref_evp_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type Theta_ref_evp_d_t: ndarray
    :param Theta_ref_cnd_d_t: 1時間平均のヒートポンプユニットの断続運転率
    :type Theta_ref_cnd_d_t: ndarray
    :param Theta_ref_SC_d_t: ヒートポンプサイクルの過冷却度（℃）
    :type Theta_ref_SC_d_t: ndarray
    :param Theta_ref_SH_d_t: ヒートポンプサイクルの過熱度（℃）
    :type Theta_ref_SH_d_t: ndarray
    :return: 日付dの時刻tにおける圧縮機の圧縮効率 (3a)
    :rtype: ndarray
    """
    # Kμh0：圧縮機の圧縮効率を求める式の係数 (-) (3b)
    K_Mu_h_0 = get_K_Mu_h_0()

    # Kμh1：圧縮機の圧縮効率を求める式の係数 (-) (3b)
    K_Mu_h_1 = get_K_Mu_h_1()

    # Kμh2：圧縮機の圧縮効率を求める式の係数 (-) (3b)
    K_Mu_h_2 = get_K_Mu_h_2()

    # 日付dの時刻tにおけるヒートポンプサイクルの理論暖房効率(-) 4章8節付録A(1)
    e_ref_H_th_d_t = calc_e_ref_H_th(Theta_ref_evp_d_t, Theta_ref_cnd_d_t, Theta_ref_SC_d_t, Theta_ref_SH_d_t)

    # 日付dの時刻tにおける圧縮機の圧縮効率 (3a)
    Mu_d_t = K_Mu_h_2 * (e_ref_H_th_d_t ** 2) + K_Mu_h_1 * e_ref_H_th_d_t + K_Mu_h_0
    Mu_d_t[e_ref_H_th_d_t > 10] = K_Mu_h_2 * (10 ** 2) + K_Mu_h_1 * 10 + K_Mu_h_0

    return Mu_d_t


def get_K_Mu_h_0():
    """ Kμh0：圧縮機の圧縮効率を求める式の係数 (-) (3b)

    :return: Kμh0：圧縮機の圧縮効率を求める式の係数 (-) (3b)
    :rtype: float
    """
    return -0.430363368361459


def get_K_Mu_h_1():
    """ Kμh1：圧縮機の圧縮効率を求める式の係数 (-) (3b)

    :return: Kμh1：圧縮機の圧縮効率を求める式の係数 (-) (3b)
    :rtype: float
    """
    return 0.698531770387591


def get_K_Mu_h_2():
    """ Kμh2：圧縮機の圧縮効率を求める式の係数 (-) (3b)

    :return: Kμh2：圧縮機の圧縮効率を求める式の係数 (-) (3b)
    :rtype: float
    """
    return 0.0100164335768507

日付$d$の時刻$t$におけるヒートポンプサイクルの理論暖房効率$e_{ref,H,th,d,t}$が 10 を超える場合は$e_{ref,H,th,d,t}$は 10 とする。

If the theoretical heating efficiency $e_{ref,H,th,d,t}$ of the heat pump cycle at time $t$ on date $d$ exceeds 10, $e_{ref,H,th,d,t}$ is set to 10.

$$\large
\theta_{ref,evp,d,t} = max(k_{envh0}+k_{envh1} \times \theta_{gsRW,d\_ave,d} +k_{envh2} \times qr_{out,H,hs,d,t} +k_{envh12}  \times \theta_{gsRW,d\_ave,d} \times qr_{out,H,hs,d,t}, -50)
$$

$$\large
\left\{ \begin{array}{ll}
    k_{envh0}= −2.95315205817646\\  
    k_{envh1}= 0.915893610614308\\  
    k_{envh2}= −11.8319776584846\\  
    k_{envh12}= 0.29704275467947\\  
    \end{array}
\right.
$$

$$\large
\theta_{ref,cnd,d,t} = min(k_{cndh0}+k_{cndh1} \times \theta_{SW,d,t} +k_{cndh2} \times qr_{out,H,hs,d,t} +k_{cndh12}  \times \theta_{SW,d,t} \times qr_{out,H,hs,d,t}, 65)
$$

$$\large
\left\{ \begin{array}{ll}
    k_{cndh0}= 3.6105623002886\\  
    k_{cndh1}= 0.930136847064537\\  
    k_{cndh2}= 0.494024927234563\\  
    k_{cndh12}= 0.00770898511188855\\  
    \end{array}
\right.
$$

$$\large
\theta_{ref,SC,d,t} = max(k_{sch0}+k_{sch1} \times \theta_{SW,d,t} +k_{sch2} \times qr_{out,H,hs,d,t}, 0)
$$

$$\large
\left\{ \begin{array}{ll}
    k_{sch0}= −4.02655782981397\\  
    k_{sch1}= 0.0894330494418674\\  
    k_{sch2}= 14.3457831669162\\  
    \end{array}
\right.
$$

$$\large
\theta_{ref,SH,d,t} = max(k_{shh0}+k_{shh1} \times qr_{out,H,hs,d,t} +k_{shh2} \times \theta_{gsRW,d\_ave,d}, 0)
$$

$$\large
\left\{ \begin{array}{ll}
    k_{shh0}= 0.819643791668597\\  
    k_{shh1}= 2.99282570323758\\  
    k_{shh2}= −0.0762659183765636\\  
    \end{array}
\right.
$$

$qr_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の最大暖房能力に対する平均負荷率(-) / Average load factor for maximum heating capacity of the heat source unit for hot water heating at time $t$ on date $d$  
$\theta_{gsRW,d\_ave,d}$：日付$d$における地中熱交換器からの戻り熱源水の日平均温度(℃) / Daily mean temperature of return heat source water from underground heat exchanger at date $d$  
$\theta_{ref,evp,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの蒸発温度(℃) / Evaporation temperature of heat pump cycle at time $t$ on date $d$  
$\theta_{ref,cnd,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの凝縮温度(℃) / Condensation temperature of heat pump cycle at time $t$ on date $d$  
$\theta_{ref,SC,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの過冷却度(℃) / Degree of subcooling of heat pump cycle at time $t$ on date $d$  
$\theta_{ref,SH,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクルの過熱度(℃) / Superheat of heat pump cycle at time $t$ on date $d$  
$\theta_{SW,d,t}$：日付$d$の時刻$t$における往き温水温度(℃) / Outgoing hot water temperature at time $t$ on date $d$  
$k_{cndh0},k_{cndh1},k_{cndh2},k_{cndh12}$：凝縮温度を計算する式の係数(-) / Coefficient of formula for calculating condensation temperature  
$k_{envh0},k_{envh1},k_{envh2},k_{envh12}$：蒸発温度を計算する式の係数(-) / Coefficient of formula for calculating evaporation temperature  
$k_{sch0},k_{sch1},k_{sch2}$：過冷却度を計算する式の係数(-) / Coefficient of formula for calculating degree of supercooling  
$k_{shh0},k_{shh1},k_{shh2}$：過熱度を計算する式の係数(-) / Coefficient of formula for calculating superheat  

In [ ]:
def calc_Theta_ref_evp_d_t(qr_out_H_hs_d_t, Theta_gsRW_d_ave_d):
    """ 日付dの時刻tにおけるヒートポンプサイクルの蒸発温度（℃） (4a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :param Theta_gsRW_d_ave_d: 日付dにおける地中熱交換器からの戻り熱源水の日平均温度（℃）
    :type Theta_gsRW_d_ave_d: ndarray
    :return: 日付dの時刻tにおけるヒートポンプサイクルの蒸発温度（℃） (4a)
    :rtype:
    """
    # Kevph0：蒸発温度を計算する式の係数 (-) (4b)
    K_evp_h_0 = get_K_evp_h_0()

    # Kevph1：蒸発温度を計算する式の係数 (-) (4b)
    K_evp_h_1 = get_K_evp_h_1()

    # Kevph2：蒸発温度を計算する式の係数 (-) (4b)
    K_evp_h_2 = get_K_evp_h_2()

    # Kevph12：蒸発温度を計算する式の係数 (-) (4b)
    K_evp_h_12 = get_K_evp_h_12()

    # 日付dの時刻tにおけるヒートポンプサイクルの蒸発温度（℃） (4a)
    Theta_ref_evp_d_t = np.clip(K_evp_h_0 + K_evp_h_1 * np.repeat(Theta_gsRW_d_ave_d, 24)
                                + K_evp_h_2 * qr_out_H_hs_d_t
                                + K_evp_h_12 * np.repeat(Theta_gsRW_d_ave_d, 24) * qr_out_H_hs_d_t, -50, None)

    return Theta_ref_evp_d_t


def get_K_evp_h_0():
    """ Kevph0：蒸発温度を計算する式の係数 (-) (4b)

    :return: Kevph0：蒸発温度を計算する式の係数
    :rtype: float
    """

    return -2.95315205817646


def get_K_evp_h_1():
    """ Kevph1：蒸発温度を計算する式の係数 (-) (4b)

    :return: Kevph1：蒸発温度を計算する式の係数
    :rtype: float
    """
    return 0.915893610614308


def get_K_evp_h_2():
    """ Kevph2：蒸発温度を計算する式の係数 (-) (4b)

    :return: Kevph2：蒸発温度を計算する式の係数
    :rtype: float
    """
    return -11.8319776584846


def get_K_evp_h_12():
    """ Kevph12：蒸発温度を計算する式の係数 (-) (4b)

    :return: Kevph12：蒸発温度を計算する式の係数
    :rtype: float
    """
    return 0.29704275467947


def calc_Theta_ref_cnd_d_t(qr_out_H_hs_d_t, Theta_SW_d_t):
    """ 日付dの時刻tにおけるヒートポンプサイクルの凝縮温度（℃）(5a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t:
    :param Theta_SW_d_t: 日付dの時刻tにおける往き温水温度（℃）
    :type Theta_SW_d_t: ndarray
    :return: 日付dの時刻tにおけるヒートポンプサイクルの凝縮温度（℃）
    :rtype: ndarray
    """
    # Kcndh0：凝縮温度を計算する式の係数 (-) (5b)
    K_cnd_h_0 = get_K_cnd_h_0()

    # Kcndh1：凝縮温度を計算する式の係数 (-) (5b)
    K_cnd_h_1 = get_K_cnd_h_1()

    # Kcndh2：凝縮温度を計算する式の係数 (-) (5b)
    K_cnd_h_2 = get_K_cnd_h_2()

    # Kcndh12：凝縮温度を計算する式の係数 (-) (5b)
    K_cnd_h_12 = get_K_cnd_h_12()

    # 日付dの時刻tにおけるヒートポンプサイクルの蒸発温度（℃） (5a)
    Theta_ref_cnd_d_t = np.clip(K_cnd_h_0 + K_cnd_h_1 * Theta_SW_d_t
                                + K_cnd_h_2 * qr_out_H_hs_d_t
                                + K_cnd_h_12 * Theta_SW_d_t * qr_out_H_hs_d_t, None, 65)

    return Theta_ref_cnd_d_t


def get_K_cnd_h_0():
    """ Kcndh0：凝縮温度を計算する式の係数 (-) (5b)

    :return: Kcndh0：凝縮温度を計算する式の係数
    :rtype: float
    """
    return 3.6105623002886


def get_K_cnd_h_1():
    """ Kcndh1：凝縮温度を計算する式の係数 (-) (5b)

    :return: Kcndh1：凝縮温度を計算する式の係数
    :rtype: float
    """
    return 0.930136847064537


def get_K_cnd_h_2():
    """ Kcndh2：凝縮温度を計算する式の係数 (-) (5b)

    :return: Kcndh2：凝縮温度を計算する式の係数
    :rtype: float
    """
    return 0.494024927234563


def get_K_cnd_h_12():
    """ Kcndh12：凝縮温度を計算する式の係数 (-) (5b)

    :return: Kcndh12：凝縮温度を計算する式の係数
    :rtype: ndarray
    """
    return 0.00770898511188855


def calc_Theta_ref_SC_d_t(qr_out_H_hs_d_t, Theta_SW_d_t):
    """ 日付dの時刻tにおけるヒートポンプサイクルの過冷却度（℃）(6a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :param Theta_SW_d_t: 往き温水温度 (℃)
    :type Theta_SW_d_t: ndarray
    :return: 日付dの時刻tにおけるヒートポンプサイクルの過冷却度（℃）
    :rtype: ndarray
    """
    # Ksch0：過冷却度を計算する式の係数 (-) (6b)
    K_sc_h_0 = get_K_sc_h_0()

    # Ksch1：過冷却度を計算する式の係数 (-) (6b)
    K_sc_h_1 = get_K_sc_h_1()

    # Ksch2：過冷却度を計算する式の係数 (-) (6b)
    K_sc_h_2 = get_K_sc_h_2()

    # 日付dの時刻tにおけるヒートポンプサイクルの過冷却度（℃） (6a)
    Theta_ref_SC_d_t = np.clip(K_sc_h_0 + K_sc_h_1 * Theta_SW_d_t + K_sc_h_2 * qr_out_H_hs_d_t, 0, None)

    return Theta_ref_SC_d_t


def get_K_sc_h_0():
    """ Ksch0：過冷却度を計算する式の係数(-) (6b)

    :return: Ksch0：過冷却度を計算する式の係数(-) (6b)
    :rtype: float
    """
    return -4.02655782981397


def get_K_sc_h_1():
    """ Ksch1：過冷却度を計算する式の係数 (-) (6b)

    :return: Ksch1：過冷却度を計算する式の係数
    :rtype: float
    """
    return 0.0894330494418674


def get_K_sc_h_2():
    """ Ksch2：過冷却度を計算する式の係数 (-) (6b)

    :return: Ksch2：過冷却度を計算する式の係数
    :rtype: float
    """
    return 14.3457831669162


def calc_Theta_ref_SH_d_t(qr_out_H_hs_d_t, Theta_gsRW_d_ave_d):
    """ 日付dの時刻tにおけるヒートポンプサイクルの過熱度（℃）(7a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :param Theta_gsRW_d_ave_d: 日付dにおける地中熱交換器からの戻り熱源水の日平均温度（℃）
    :type Theta_gsRW_d_ave_d: ndarray
    :return: 日付dの時刻tにおけるヒートポンプサイクルの過熱度（℃）
    :rtype: ndarray
    """
    # Kshh0：過熱度を計算する式の係数 (-) (7b)
    K_sh_h_0 = get_K_sh_h_0()

    # Kshh1：過熱度を計算する式の係数 (-) (7b)
    K_sh_h_1 = get_K_sh_h_1()

    # Kshh2：過熱度を計算する式の係数 (-) (7b)
    K_sh_h_2 = get_K_sh_h_2()

    # 日付dの時刻tにおけるヒートポンプサイクルの過熱度（℃） (7a)
    Theta_ref_SC_d_t = np.clip(K_sh_h_0 + K_sh_h_1 * qr_out_H_hs_d_t + K_sh_h_2 * np.repeat(Theta_gsRW_d_ave_d, 24), 0, None)

    return Theta_ref_SC_d_t


def get_K_sh_h_0():
    """ Kshh0：過熱度を計算する式の係数(-) (7b)

    :return: Kshh0：過熱度を計算する式の係数
    :rtype: float
    """
    return 0.819643791668597


def get_K_sh_h_1():
    """ Kshh1：過熱度を計算する式の係数 (-) (7b)

    :return: Kshh1：過熱度を計算する式の係数 (-)
    :rtype: float
    """
    return 2.99282570323758


def get_K_sh_h_2():
    """ Kshh2：過熱度を計算する式の係数 (-) (7b)

    :return: Kshh2：過熱度を計算する式の係数 (-)
    :rtype:
    """
    return -0.0762659183765636

### N.5 ポンプの消費電力量 / Pump power consumption

$$\large
E_{pump,hs,d,t} = E_{pump,SW,d,t} + E_{pump,gsRW,d,t}
$$

$E_{pump,hs,d,t}$：日付$d$の時刻$t$における1 時間当たりのポンプの消費電力量(kWh/h) / Hourly pump power consumption at time $t$ on date $d$  
$E_{pump,gsRW,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源水ポンプの消費電力量(kWh/h) / Power consumption of heat source water pump per hour at time $t$ on date $d$  
$E_{pump,SW,d,t}$：日付$d$の時刻$t$における1 時間当たりの送水ポンプの消費電力量(kWh/h) / Electricity consumption of water pump per hour at time $t$ on date $d$  


In [ ]:
def get_E_pump_hs_d_t(E_pump_SW_d_t, E_pump_gsRW_d_t):
    """ 日付dの時刻tにおける1時間当たりのポンプの消費電力量 (8)

    :param E_pump_SW_d_t: 日付dの時刻tにおける1時間当たりの送水ポンプの消費電力量（kWh/h）
    :type E_pump_SW_d_t: ndarray
    :param E_pump_gsRW_d_t: 日付dの時刻tにおける1時間当たりの熱源水ポンプの消費電力量（kWh/h）
    :type E_pump_gsRW_d_t: ndarray
    :return: 日付dの時刻tにおける1時間当たりのポンプの消費電力量（kWh/h）
    :rtype: ndarray
    """

    # 日付dの時刻tにおける1時間当たりのポンプの消費電力量 (8)
    E_pump_hs_d_t = E_pump_SW_d_t + E_pump_gsRW_d_t

    return E_pump_hs_d_t

$$\large
E_{pump,SW,d,t}= a_{pump,SW} \times qr_{out,H,hs,d,t}+ b_{pump,SW} \times {qr_{out,H,hs,d,t}}^2
$$

$$\large
\left\{ \begin{array}{ll}
    a_{pump,SW}= 0.041972403\\  
    b_{pump,SW}= 0.104478967\\  
    \end{array}
\right.
$$

$$\large
E_{pump,gsRW,d,t}= a_{pump,gsRW} \times qr_{out,H,hs,d,t}+ b_{pump,gsRW} \times {qr_{out,H,hs,d,t}}^2
$$

$$\large
\left\{ \begin{array}{ll}
    a_{pump,gsRW}= 0.062196275\\  
    b_{pump,gsRW}= 0.071756474\\  
    \end{array}
\right.
$$

$E_{pump,SW,d,t}$：日付$d$の時刻$t$における1 時間当たりの送水ポンプの消費電力量(kWh/h) / Electricity consumption of water pump per hour at time $t$ on date $d$  
$E_{pump,gsRW,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源水ポンプの消費電力量(kWh/h) / Energy consumption of heat source water pump per hour at time $t$ on date $d$  
$qr_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の最大暖房能力に対する平均負荷率(-) / Average load factor for maximum heating capacity of the heat source unit for hot water heating at time $t$ on date $d$  
$a_{pump,SW},b_{pump,SW}$：送水ポンプの消費電力量を計算する式の係数(-) / Coefficient of the formula for calculating the power consumption of the water pump  
$a_{pump,gsRW},b_{pump,gsRW}$：熱源水ポンプの消費電力量を計算する式の係数(-) / Coefficients of formula for calculating power consumption of heat source water pump  

In [ ]:
def calc_E_pump_SW_d_t(qr_out_H_hs_d_t):
    """ 日付dの時刻tにおける1時間当たりの送水ポンプの消費電力量（kWh/h） (9a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :return: 日付dの時刻tにおける1時間当たりの送水ポンプの消費電力量（kWh/h）
    :rtype: ndarray
    """
    # apump,SW：送水ポンプの消費電力量を計算する式の係数 (-) (9b)
    a_pump_SW = get_a_pump_SW()

    # bpump,SW：送水ポンプの消費電力量を計算する式の係数 (-) (9b)
    b_pump_SW = get_b_pump_SW()

    # 日付dの時刻tにおける1時間当たりの送水ポンプの消費電力量（kWh/h） (9a)
    E_pump_SW_d_t = a_pump_SW * qr_out_H_hs_d_t + b_pump_SW * (qr_out_H_hs_d_t ** 2)

    return E_pump_SW_d_t


def get_a_pump_SW():
    """ apump,SW：送水ポンプの消費電力量を計算する式の係数 (-) (9b)

    :return: apump,SW：送水ポンプの消費電力量を計算する式の係数 (-)
    :rtype: float
    """
    return 0.041972403


def get_b_pump_SW():
    """ bpump,SW：送水ポンプの消費電力量を計算する式の係数 (-) (9b)

    :return: bpump,SW：送水ポンプの消費電力量を計算する式の係数 (-)
    :rtype: float
    """
    return 0.104478967


def calc_E_pump_gsRW_d_t(qr_out_H_hs_d_t):
    """ 日付dの時刻tにおける1時間当たりの熱源水ポンプの消費電力量（kWh/h） (10a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :return: 日付dの時刻tにおける1時間当たりの熱源水ポンプの消費電力量（kWh/h）
    :rtype: ndarray
    """
    # apump,gsRW：熱源水ポンプの消費電力量を計算する式の係数 (-) (10b)
    a_pump_gsRW = get_a_pump_gsRW()

    # bpump,gsRW：熱源水ポンプの消費電力量を計算する式の係数 (-) (10b)
    b_pump_gsRW = get_b_pump_gsRW()

    # 日付dの時刻tにおける1時間当たりの熱源水ポンプの消費電力量（kWh/h） (10a)
    E_pump_gsRW_d_t = a_pump_gsRW * qr_out_H_hs_d_t + b_pump_gsRW * (qr_out_H_hs_d_t ** 2)

    return E_pump_gsRW_d_t


def get_a_pump_gsRW():
    """ apump,gsRW：熱源水ポンプの消費電力量を計算する式の係数 (-) (10b)

    :return: apump,gsRW：熱源水ポンプの消費電力量を計算する式の係数 (-)
    :rtype: float
    """
    return 0.062196275


def get_b_pump_gsRW():
    """ bpump,gsRW：熱源水ポンプの消費電力量を計算する式の係数 (-) (10b)

    :return: bpump,gsRW：熱源水ポンプの消費電力量を計算する式の係数 (-)
    :rtype:
    """
    return 0.071756474

### N.6 補機の消費電力量 / Auxiliary equipment power consumption

$$\large
E_{aux,hs,d,t} = k_{auxh1} \times qr_{out,H,hs,d,t} + k_{auxh0}
$$

$$\large
\left\{ \begin{array}{ll}
    k_{auxh1}= 0.0433205551083371\\  
    k_{auxh2}= 0.0173758330059922\\  
    \end{array}
\right.
$$

$E_{aux,hs,d,t}$：日付$d$の時刻$t$における1 時間当たりの補機の消費電力量(kWh/h) / Electric power consumption of auxiliary equipment per hour at time $t$ on date $d$  
$qr_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の最大暖房能力に対する平均負荷率(-) / Average load factor for maximum heating capacity of the heat source unit for hot water heating at time $t$ on date $d$  
$k_{auxh1},k_{auxh0}$：補機の消費電力を計算する式の係数(-) / Coefficient of formula for calculating power consumption of auxiliary equipment  

In [ ]:
def calc_E_aux_hs_d_t(qr_out_H_hs_d_t):
    """ 日付dの時刻tにおける1時間当たりの補機の消費電力量（kWh/h） (11a)

    :param qr_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type qr_out_H_hs_d_t: ndarray
    :return: 日付dの時刻tにおける1時間当たりの補機の消費電力量（kWh/h）
    :rtype: ndarray
    """
    # kauxh0：補機の消費電力量を計算する式の係数 (-) (11b)
    kauxh0 = get_kauxh0()

    # kauxh1：補機の消費電力量を計算する式の係数 (-) (11b)
    kauxh1 = get_kauxh1()

    # 日付dの時刻tにおける1時間当たりの熱源水ポンプの消費電力量（kWh/h） (11a)
    E_aux_hs_d_t = kauxh1 * qr_out_H_hs_d_t + kauxh0

    return E_aux_hs_d_t


def get_kauxh0():
    """ kauxh0：補機の消費電力量を計算する式の係数 (-) (11b)

    :return: kauxh0：補機の消費電力量を計算する式の係数 (-)
    :rtype: float
    """
    return 0.0433205551083371


def get_kauxh1():
    """ kauxh1：補機の消費電力量を計算する式の係数 (-) (11b)

    :return: kauxh1：補機の消費電力量を計算する式の係数 (-)
    :rtype: float
    """
    return 0.0173758330059922

### N.7 温水暖房用熱源機の最大暖房能力に対する平均負荷率 / Average load factor for the maximum heating capacity of the heat source unit for hot water heating

$$\large
qr_{out,H,hs,d,t} = \frac{q_{out,H,hs,d,t}\times 10^3}{q_{max,H,hs,JRA}}
$$

$q_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の平均暖房出力(kW) / Average heating output of heat source machine for hot water heating at time $t$ on date $d$  
$q_{max,H,hs,JRA}$：地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W) / Maximum heating capacity after correction by return heat source water temperature and water temperature from the ground  
$qr_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の最大暖房能力に対する平均負荷率(-) / Average load factor for maximum heating capacity of the heat source unit for hot water heating at time $t$ on date $d$  


In [ ]:
def get_qr_out_H_hs_d_t(q_out_H_hs_d_t, q_max_H_hs_JRA):
    """ 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率（-） (12)

    :param q_out_H_hs_d_t: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率(-)
    :type q_out_H_hs_d_t: ndarray
    :param q_max_H_hs_JRA: 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W)
    :type q_max_H_hs_JRA: ndarray
    :return: 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率（-）
    :rtype: ndarray
    """
    # 日付dの時刻tにおける温水暖房用熱源機の最大暖房能力に対する平均負荷率（-） (12)
    qr_out_H_hs_d_t = (q_out_H_hs_d_t * 10 ** 3) / q_max_H_hs_JRA

    return qr_out_H_hs_d_t

### N.8 温水暖房用熱源機の平均暖房出力 / Average heating output of heat source for hot water heating

$$\large
q_{out,H,hs,d,t} = Q_{out,H,hs,d,t} \div 3600 \times 10^3
$$

$Q_{out,H,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h) / Heating output of heat source machine for hot water heating per hour at time $t$ on date $d$  
$q_{out,H,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の平均暖房出力(kW) / Average heating output of heat source machine for hot water heating at time $t$ on date $d$  


In [ ]:
def get_q_out_H_hs_d_t(Q_out_H_hs_d_t):
    """ 日付dの時刻tにおける温水暖房用熱源機の平均暖房出力（kW） (13)

    :param Q_out_H_hs_d_t: 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の暖房出力(MJ/h)
    :type Q_out_H_hs_d_t: ndarray
    :return: 日付dの時刻tにおける温水暖房用熱源機の平均暖房出力（kW）
    :rtype: ndarray
    """
    # 日付dの時刻tにおける温水暖房用熱源機の平均暖房出力（kW）(13)
    q_out_H_hs_d_t = Q_out_H_hs_d_t / 3600 * 10 ** 3

    return q_out_H_hs_d_t

$$\large
Q_{out,H,hs,d,t} = min(Q_{dmd,H,hs,d,t},Q_{max,H,hs,d,t})
$$

$Q_{dmd,H,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの温水暖房用熱源機の温水熱需要(MJ/h) / Hourly hot water heat demand of hot water heating heat source machine at time $t$ on date $d$  
$q_{max,H,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h) / Maximum heating output of heat source machine for hot water heating per hour at time $t$ on date $d$  


In [ ]:
def get_Q_out_H_hs_d_t(Q_dmd_H_hs_d_t, Q_max_H_hs_d_t):
    """ 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の暖房出力（MJ/h）(14)

    :param Q_dmd_H_hs_d_t: 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の温水熱需要(MJ/h)
    :type Q_dmd_H_hs_d_t: ndarray
    :param Q_max_H_hs_d_t: 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h)
    :type Q_max_H_hs_d_t: ndarray
    :return: 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の暖房出力（MJ/h）
    :rtype: ndarray
    """
    # 日付dの時刻tにおける温水暖房用熱源機の平均暖房出力（kW）(14)
    return np.min([Q_dmd_H_hs_d_t, Q_max_H_hs_d_t], axis=0)

$$\large
Q_{max,H,hs,d,t} = (−0.005635139785329 \times \theta_{SW,d,t} + 0.0258983299329793 \times max(min(\theta_{gsRW,d\_ave,d},20),0)+0.836930642418471) \times q_{max,H,hs,JRA} \times 3600 \times 10^{-6}
$$

$Q_{max,H,hs,d,t}$：日付$d$の時刻$t$における 1 時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h) / Maximum heating output of the heat source machine for hot water heating per hour at time $t$ on date $d$  
$q_{max,H,hs,JRA}$：地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W) / Maximum heating capacity after correction by the return heat source water temperature from the ground and the sending water temperature  
$\theta_{SW,d,t}$：日付$d$の時刻$t$における往き温水温度(℃) / Outgoing hot water temperature at time $t$ on date $d$  
$\theta_{gsRW,d\_ave,d}$：日付$d$における地中熱交換器からの戻り熱源水の日平均温度(℃) / Daily mean temperature of return heat source water from underground heat exchangers at date $d$  


In [ ]:
def calc_Q_max_H_hs_d_t(Theta_SW_d_t, Theta_ex_d_Ave_d, Theta_ex_H_Ave, Theta_ex_a_Ave, q_max_hs, L_H_x_t_i, L_CS_x_t_i, L_CL_x_t_i, HeatExchangerType):
    """ 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h) (15)

    :param Theta_SW_d_t: 往き温水温度 (℃)
    :type Theta_SW_d_t: ndarray
    :param Theta_ex_d_Ave_d: 日付dにおける日平均外気温度 (℃)
    :type Theta_ex_d_Ave_d: ndarray
    :param Theta_ex_H_Ave: 暖房期における期間平均外気温度（℃）
    :type Theta_ex_H_Ave: float
    :param Theta_ex_a_Ave: 年平均外気温度 (℃)
    :type Theta_ex_a_Ave: float
    :param q_max_hs: 熱源機の最大暖房能力 ⒲
    :type q_max_hs: float
    :param L_H_x_t_i: 暖冷房区画iの1時間当たりの暖房負荷
    :type L_H_x_t_i: ndarray
    :param L_CS_x_t_i: 日付dの時刻tにおける暖冷房区画iの冷房顕熱負荷 (MJ/h)
    :type L_CS_x_t_i: ndarray
    :param L_CL_x_t_i: 日付dの時刻tにおける暖冷房区画iの冷房潜熱負荷 (MJ/h)
    :type L_CL_x_t_i: ndarray
    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h)
    :rtype: ndarray
    """
    # ---------- 地中熱交換器からの戻り熱源水の日平均温度 ----------

    # 1日当たりの冷房全熱負荷の年間最大値（MJ/d）(20c)
    L_max_C = get_L_max_C(L_CS_x_t_i, L_CL_x_t_i)

    # 1日当たりの暖房負荷の年間最大値（MJ/d）(20b)
    L_max_H = get_L_max_H(L_H_x_t_i)

    # 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）(20a)
    R_L_max = get_R_L_max(L_max_H, L_max_C)

    # 地中熱交換器からの戻り熱源水温度を求める式の係数（-）(19)
    K_gsRW_H = calc_K_gsRW_H(R_L_max, HeatExchangerType)

    # 暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差（℃）(18)
    Delta_Theta_gsRW_H = calc_Delta_Theta_gsRW_H(R_L_max, HeatExchangerType)

    # 日付dにおける地中熱交換器からの戻り熱源水の日平均温度（℃）(17)
    Theta_gsRW_d_ave_d = get_Theta_gsRW_d_ave_d(K_gsRW_H, Theta_ex_d_Ave_d, Theta_ex_H_Ave, Theta_ex_a_Ave,
                                                Delta_Theta_gsRW_H)

    # ---------- 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W) (16) ----------
    q_max_H_hs_JRA = calc_q_max_H_hs_JRA(q_max_hs)

    # ---------- 日付dの時刻tにおける1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h)(15) ----------
    Q_max_H_hs_d_t = (-0.005635139785329 * Theta_SW_d_t
                      + 0.0258983299329793 * np.clip(np.repeat(Theta_gsRW_d_ave_d, 24), 0, 20) + 0.836930642418471) * q_max_H_hs_JRA * 3600 * 10 ** (-6)

    return Q_max_H_hs_d_t

### N.9 温水暖房用熱源機内の平均放熱損失 / Average heat radiation loss in heat source for hot water heating

日付$d$の時刻$t$における温水暖房用熱源機内の平均放熱損失$q_{loss,H,hs,d,t}$ は、0とする。

The average heat dissipation loss $q_{loss,H,hs,d,t}$ in the heat source unit for hot water heating at time $t$ on date $d$ shall be zero.

In [ ]:
def get_q_loss_H_hs_d_t():
    """ 日付dの時刻tにおける温水暖房用熱源機内の平均放熱損失（kW）

    :return: 日付dの時刻tにおける温水暖房用熱源機内の平均放熱損失（kW）
    :rtype: float
    """
    return 0

### N.10 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力 / Maximum heating capacity after correction by the return heat source water temperature from the ground and the sending water temperature

$$\large
q_{max,H,hs,JRA} = q_{max,hs} \times f_{crated}
$$

$q_{max,hs}$：温水暖房用熱源機の最大暖房能力(W) / Maximum heating capacity of heat source unit for hot water heating  
$q_{max,H,hs,JRA}$：地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W) / Maximum heating capacity after correction by the return heat source water temperature from the ground and the sending water temperature  
$f_{crated}$：地中からの戻り熱源水温度および送水温度に関する最大暖房能力の補正係数(-) / Correction coefficient of maximum heating capacity for return heat source water temperature and water supply temperature from underground  


In [ ]:
def calc_q_max_H_hs_JRA(q_max_hs):
    """ 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W) (16)

    :param q_max_hs: 熱源機の最大暖房能力 ⒲
    :type q_max_hs:
    :return: 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W)
    :rtype:
    """
    # 地中からの戻り熱源水温度および送水温度に関する最大暖房能力の補正係数(-)
    f_crated = get_f_crated()

    # 地中からの戻り熱源水温度および送水温度による補正後の最大暖房能力(W)(16)
    q_max_H_hs_JRA = q_max_hs * f_crated

    return q_max_H_hs_JRA


def calc_q_rtd_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    """ 温水暖房用熱源機の定格能力

    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param mode_MR: 主たる居室の運転モード 'い', 'ろ', 'は'
    :type mode_MR: str
    :param mode_OR: その他の居室の運転モード 'い', 'ろ', 'は'
    :type mode_OR: str
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_MR_hwh: bool
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :type has_OR_hwh: bool
    :return: 温水暖房用熱源機の定格能力
    :rtype: float
    """
    # 付録Hに定める温水暖房用熱源機の最大能力 q_max_hs に等しい
    return appendix_H.calc_q_max_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh)

地中からの戻り熱源水温度および送水温度に関する最大暖房能力の補正係数$f_{crated}$は、1.35 とする。


The correction coefficient $f_{crated}$ of the maximum heating capacity for the return heat source water temperature and the water sending temperature from the ground is 1.35.

In [ ]:
def get_f_crated():
    """ 地中からの戻り熱源水温度および送水温度に関する最大暖房能力の補正係数(-)

    :return: 地中からの戻り熱源水温度および送水温度に関する最大暖房能力の補正係数(-)
    :rtype: float
    """
    return 1.35

### N.11 地中熱交換器からの戻り熱源水の日平均温度 / Daily average temperature of return heat source water from underground heat exchanger

$$\large
\theta_{gsRW,d\_ave,d} = k_{gsRW,H}(\theta_{ex,d\_Ave,d}-\theta_{ex,H\_Ave})+\theta_{ex,a\_Ave}+\Delta\theta_{gsRW,H}
$$

$\Delta\theta_{gsRW,H}$：暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差(℃) / Difference between the annual mean temperature and the annual mean outside temperature of the return heat source water from the underground heat exchanger during the heating period  
$\theta_{ex,a\_Ave}$：年平均外気温度(℃) / Average annual outside temperature  
$\theta_{ex,d\_Ave,d}$：日付$d$における日平均外気温度(℃) / Average daily outside temperature at date $d$  
$\theta_{ex,H\_Ave}$：暖房期における期間平均外気温度(℃) / Average temperature of outside air during heating period  
$\theta_{gsRW,d\_ave,d}$：日付$d$における地中熱交換器からの戻り熱源水の日平均温度(℃) / Daily mean temperature of return heat source water from underground heat exchangers at date $d$  
$k_{gsRW,H}$：地中熱交換器からの戻り熱源水温度を求める式の係数(-) / Coefficients of the equation for the return heat source water temperature from the underground heat exchanger  


In [ ]:
def get_Theta_gsRW_d_ave_d(K_gsRW_H, Theta_ex_d_Ave_d, Theta_ex_H_Ave, Theta_ex_a_Ave, Delta_Theta_gsRW_H):
    """ 日付dにおける地中熱交換器からの戻り熱源水の日平均温度(℃) (17)

    :param K_gsRW_H: K_gsRW_H: 地中熱交換器からの戻り熱源水温度を求める式の係数(-)
    :type K_gsRW_H: float
    :param Theta_ex_d_Ave_d: 日付dにおける日平均外気温度 (℃)
    :type Theta_ex_d_Ave_d: ndarray
    :param Theta_ex_H_Ave: 暖房期における期間平均外気温度（℃）
    :type Theta_ex_H_Ave: float
    :param Theta_ex_a_Ave: 年平均外気温度 (℃)
    :type Theta_ex_a_Ave: float
    :param Delta_Theta_gsRW_H: 暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差（℃）
    :type Delta_Theta_gsRW_H: float
    :return: 日付dにおける地中熱交換器からの戻り熱源水の日平均温度(℃)
    :rtype: ndarray
    """
    # 日付dにおける地中熱交換器からの戻り熱源水の日平均温度(℃) (17)
    Theta_gsRW_d_ave_d = K_gsRW_H * (Theta_ex_d_Ave_d - Theta_ex_H_Ave) + Theta_ex_a_Ave + Delta_Theta_gsRW_H

    return Theta_gsRW_d_ave_d

$$\large
\Delta\theta_{gsRW,H}= a_{gsRW,H}R_{L_{max}} +b_{gsRW,H}
$$

$R_{L_{max}}$：1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比(-) / The ratio of these differences to the sum of the annual maximum heating load and the maximum cooling load per day.  
$\Delta\theta_{gsRW,H}$：暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差(℃) / Difference between the annual mean temperature and the annual mean outside temperature of the return heat source water from the underground heat exchanger during the heating period  
$a_{gsRW,H}、b_{gsRW,H}$：係数(-) / coefficient  


In [ ]:
def calc_Delta_Theta_gsRW_H(R_L_max, HeatExchangerType):
    """ 暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差（℃）(18)

    :param R_L_max: 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）
    :type R_L_max: float
    :param HeatExchangerType: 地中熱交換器タイプ
    :type HeatExchangerType: str
    :return: 暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差（℃）
    :rtype: float
    """
    # 熱交換器タイプに応じた係数取得
    a_gsRW_H = get_a_gsRW_H(HeatExchangerType)
    b_gsRW_H = get_b_gsRW_H(HeatExchangerType)

    # 暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差（℃）(18)
    Delta_Theta_gsRW_H = a_gsRW_H * R_L_max + b_gsRW_H

    return Delta_Theta_gsRW_H


$$\large
k_{gsRW,H}= c_{gsRW,H}R_{L_{max}} +d_{gsRW,H}
$$

$R_{L_{max}}$：1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比(-) / The ratio of these differences to the sum of the annual maximum heating load and the maximum cooling load per day.  
$\Delta\theta_{gsRW,H}$：暖房期における地中熱交換器からの戻り熱源水の期間平均温度と年平均外気温度との差(℃) / Difference between the annual mean temperature and the annual mean outside temperature of the return heat source water from the underground heat exchanger during the heating period  
$c_{gsRW,H}、d_{gsRW,H}$：係数(-) / coefficient  


In [ ]:
def calc_K_gsRW_H(R_L_max, HeatExchangerType):
    """ 地中熱交換器からの戻り熱源水温度を求める式の係数（-）(19)

    :param R_L_max: 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）
    :type R_L_max: float
    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 地中熱交換器からの戻り熱源水温度を求める式の係数（-）(19)
    :rtype: float
    """
    # 熱交換器タイプに応じた係数取得
    c_gsRW_H = get_c_gsRW_H(HeatExchangerType)
    d_gsRW_H = get_d_gsRW_H(HeatExchangerType)

    # 地中熱交換器からの戻り熱源水温度を求める式の係数（-）(19)
    K_gsRW_H = c_gsRW_H * R_L_max + d_gsRW_H

    return K_gsRW_H

<div style="text-align: center;font-weight: bold;">表　係数<br>Table: Coefficient</div>

<table>
<thead>
<tr>
<th rowspan="2">熱交換器タイプ<br>Heat exchanger type</th>
<th colspan="4" style="text-align: center;">係数(-)<br>Coefficient</th>
</tr>
<tr>
<th>$a_{gsRW,H}$</th>
<th>$b_{gsRW,H}$</th>
<th>$c_{gsRW,H}$</th>
<th>$d_{gsRW,H}$</th>
</tr>
</thead>
<tbody>
<tr><th>1</th><td>3.1672</td><td>-0.4273</td><td>-0.0444</td><td>0.0442</td></tr>
<tr><th>2</th><td>5.9793</td><td>-1.0687</td><td>-0.1613</td><td>0.1047</td></tr>
<tr><th>3</th><td>8.3652</td><td>-1.5946</td><td>-0.2486</td><td>0.1546</td></tr>
<tr><th>4</th><td>9.9065</td><td>-2.1827</td><td>-0.3454</td><td>0.2072</td></tr>
<tr><th>5</th><td>10.2898</td><td>-2.8727</td><td>-0.3270</td><td>0.2700</td></tr>
</tbody>
</table>


In [ ]:
def get_a_gsRW_H(HeatExchangerType):
    """ 熱交換器タイプに応じた係数a_gsRW_Hの取得

    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 熱交換器タイプに応じた係数a_gsRW_Hの取得
    :rtype: float
    """

    if HeatExchangerType == '1':
        return get_table_n_3()[0][0]
    elif HeatExchangerType == '2':
        return get_table_n_3()[1][0]
    elif HeatExchangerType == '3':
        return get_table_n_3()[2][0]
    elif HeatExchangerType == '4':
        return get_table_n_3()[3][0]
    elif HeatExchangerType == '5':
        return get_table_n_3()[4][0]
    else:
        raise ValueError(HeatExchangerType)


def get_b_gsRW_H(HeatExchangerType):
    """ 熱交換器タイプに応じた係数b_gsRW_Hの取得

    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 熱交換器タイプに応じた係数b_gsRW_Hの取得
    :rtype: float
    """
    if HeatExchangerType == '1':
        return get_table_n_3()[0][1]
    elif HeatExchangerType == '2':
        return get_table_n_3()[1][1]
    elif HeatExchangerType == '3':
        return get_table_n_3()[2][1]
    elif HeatExchangerType == '4':
        return get_table_n_3()[3][1]
    elif HeatExchangerType == '5':
        return get_table_n_3()[4][1]
    else:
        raise ValueError(HeatExchangerType)


def get_c_gsRW_H(HeatExchangerType):
    """ 熱交換器タイプに応じた係数a_gsRW_Hの取得

    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 熱交換器タイプに応じた係数a_gsRW_Hの取得
    :rtype: float
    """
    if HeatExchangerType == '1':
        return get_table_n_3()[0][2]
    elif HeatExchangerType == '2':
        return get_table_n_3()[1][2]
    elif HeatExchangerType == '3':
        return get_table_n_3()[2][2]
    elif HeatExchangerType == '4':
        return get_table_n_3()[3][2]
    elif HeatExchangerType == '5':
        return get_table_n_3()[4][2]
    else:
        raise ValueError(HeatExchangerType)


def get_d_gsRW_H(HeatExchangerType):
    """ 熱交換器タイプに応じた係数b_gsRW_Hの取得

    :param HeatExchangerType: 熱交換器タイプ (-)
    :type HeatExchangerType: str
    :return: 熱交換器タイプに応じた係数b_gsRW_Hの取得
    :rtype: float
    """
    if HeatExchangerType == '1':
        return get_table_n_3()[0][3]
    elif HeatExchangerType == '2':
        return get_table_n_3()[1][3]
    elif HeatExchangerType == '3':
        return get_table_n_3()[2][3]
    elif HeatExchangerType == '4':
        return get_table_n_3()[3][3]
    elif HeatExchangerType == '5':
        return get_table_n_3()[4][3]
    else:
        raise ValueError(HeatExchangerType)


def get_table_n_3():
    """ 表N.3 係数

    :return: 表N.3 係数
    :rtype: list
    """
    table_n_3 = [
        (3.1672, -0.4273, -0.0444, 0.0442),
        (5.9793, -1.0687, -0.1613, 0.1047),
        (8.3652, -1.5946, -0.2486, 0.1546),
        (9.9065, -2.1827, -0.3454, 0.2072),
        (10.2898, -2.8727, -0.3270, 0.2700)
    ]
    return table_n_3

$$\large
R_{L_{max}}= \frac{L_{max,C}-L_{max,H}}{L_{max,C}+L_{max,H}}
$$

$$\large
L_{max,H}= max\left\{ \sum^{23}_{t=0}\sum^{12}_{i=1}L_{H,x,t,i}| 1\leq x \leq 365 \right\}
$$

$$\large
L_{max,C}= max\left\{ \sum^{23}_{t=0}\sum^{12}_{i=1}(L_{CS,x,t,i}+L_{CL,x,t,i})| 1\leq x \leq 365 \right\}
$$

$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の冷房顕熱負荷(MJ/h) / Cooling sensible heat load of heating/cooling section $i$ at time $t$ on date $d$  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の冷房潜熱負荷(MJ/h) / Cooling latent heat load of heating/cooling section $i$ at time $t$ on date $d$  
$L_{H,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の暖房負荷(MJ/h) / Heating load of heating/cooling section $i$ at time $t$ on date $d$  
$L_{max,H}$：1 日当たりの暖房負荷の年間最大値(MJ/d) / Annual maximum heating load per day  
$L_{max,C}$：1 日当たりの冷房全熱負荷の年間最大値(MJ/d) / Annual maximum total cooling heat load per day  
$R_{L_{max}}$：1 日当たりの暖房負荷の年間最大値と 1 日当たりの冷房負荷の年間最大値の和に対する、これらの差の比(-) / The ratio of these differences to the sum of the annual maximum heating load per day and the annual maximum cooling load per day.  


In [ ]:
def get_R_L_max(L_max_H, L_max_C):
    """ 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）(20a)

    :param L_max_H: 1日当たりの暖房負荷の年間最大値（MJ/d）
    :type L_max_H: float
    :param L_max_C: 1日当たりの冷房全熱負荷の年間最大値（MJ/d）
    :type L_max_C: float
    :return: 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）(20a)
    :rtype: float
    """
    # 1日当たりの暖房負荷の年間最大値と1日当たりの冷房負荷の年間最大値の和に対する、これらの差の比（-）(20a)
    R_L_max = (L_max_C - L_max_H) / (L_max_C + L_max_H)

    return R_L_max


def get_L_max_H(L_H_x_t_i):
    """ 1日当たりの暖房負荷の年間最大値（MJ/d）(20b)

    :param L_H_x_t_i: 暖冷房区画iの1時間当たりの暖房負荷
    :type L_H_x_t_i: ndarray
    :return: 1日当たりの暖房負荷の年間最大値（MJ/d）
    :rtype: float
    """
    # L_H_x_t_iは暖冷房区画毎に365日×24時間分の負荷を持った2次元配列
    # 暖冷房区画軸合算(暖冷房区画の次元をなくす)
    L_H_x_t = np.sum(L_H_x_t_i, axis=0)

    # 1次元配列を2次元配列に形状変換する
    L_H_x_t = np.reshape(L_H_x_t, (365, 24))

    # 時間軸合算
    L_H_x = np.sum(L_H_x_t, axis=1)

    # 1日当たりの暖房負荷の年間最大値（MJ/d）(20b)
    L_max_H = np.max(L_H_x)

    return L_max_H


def get_L_max_C(L_CS_x_t_i, L_CL_x_t_i):
    """ 1日当たりの冷房全熱負荷の年間最大値（MJ/d）(20c)

    :param L_CS_x_t_i: 日付dの時刻tにおける暖冷房区画iの冷房顕熱負荷 (MJ/h)
    :type L_CS_x_t_i: ndarray
    :param L_CL_x_t_i: 日付dの時刻tにおける暖冷房区画iの冷房潜熱負荷 (MJ/h)
    :type L_CL_x_t_i: ndarray
    :return: 1日当たりの冷房全熱負荷の年間最大値（MJ/d）
    :rtype: float
    """
    # 暖冷房区画軸合算(暖冷房区画の次元をなくす)
    L_CS_x_t = np.sum(L_CS_x_t_i, axis=0)
    L_CL_x_t = np.sum(L_CL_x_t_i, axis=0)

    # L_CS_x_tとL_CL_x_tの要素同士を足す
    L_C_x_t = L_CS_x_t + L_CL_x_t

    # 1次元配列を2次元配列に形状変換する
    L_C_x_t = np.reshape(L_C_x_t, (365, 24))

    # 時間軸合算
    L_C_x = np.sum(L_C_x_t, axis=1)

    # 1日当たりの冷房全熱負荷の年間最大値（MJ/d）(20c)
    L_max_C = np.max(L_C_x)

    return L_max_C

### N.12 平均外気温度 / Average outside temperature

省略

ommision

### N.13 暖房負荷・冷房負荷 / Heating load/cooling load

省略

ommision